In [ ]:
!pip install git+https://github.com/huggingface/transformers -qqq
!pip install datasets -qqq
!pip install bitsandbytes -qqq
!pip install huggingface_hub -qqq
!pip install peft -qqq
!pip install accelerate -qqq
!pip install trl -qqq
!pip install wandb -qqq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from peft import AutoPeftModelForCausalLM

In [ ]:
# Set CUDA devices
torch.cuda.is_available()
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Load Mistral model
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    use_cache=False,
    device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Apply LoRA
lora_config = LoraConfig(
    r=64,
    lora_alpha=128,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

base_model = prepare_model_for_kbit_training(base_model)
model = get_peft_model(base_model, lora_config)


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
# Print trainable parameters
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print_trainable_parameters(model)

trainable params: 27262976 || all params: 3779334144 || trainable%: 0.7213698223345028


In [ ]:
# Load dataset
dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(dataset_name)
dataset_subset = dataset["train"]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
def generate_prompt(example, return_response=True) -> str:
    INTRO = "<s>[INST] Below is a real-life conversation:"
    END = "Instruction: Continue the conversation as if you were one of the participants. [/INST]</s>"

    # Assuming 'dialogue' is the key for the conversation text in your dataset
    dialogues = example['dialogue']

    # Process each dialogue in the batch
    formatted_prompts = []
    for dialogue in dialogues:
        lines = dialogue.split('#')[1:]  # Split lines using '#' as delimiter, ignore the first empty element
        conversation_lines = [f"{line.strip()}" for i, line in enumerate(lines)]
        formatted_prompt = "\n".join([INTRO] + conversation_lines + [END])
        formatted_prompts.append(formatted_prompt)

    return formatted_prompts


In [ ]:
training_args = TrainingArguments(
    "talktactapp/GPmodelMistral",
    per_device_train_batch_size=1,  # Best practice: https://huggingface.co/docs/transformers/main/main_classes/quantization#tips-and-best-practices
    gradient_accumulation_steps=4,  # Powers of 2.
    learning_rate=2e-4,
    max_grad_norm=1.0,
    max_steps=20,
    lr_scheduler_type="linear",
    warmup_steps=5,
    fp16=True,
    logging_strategy="steps",
    logging_steps=1,
    save_strategy="steps",
    save_steps=10,
    optim="paged_adamw_8bit",
    report_to="wandb",
    push_to_hub= True
)

In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_subset,
    peft_config=lora_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    formatting_func=generate_prompt,
    args=training_args,
)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

Step,Training Loss
1,2.818900
2,2.881900
3,2.593100
4,2.298600
5,2.263600
6,1.986800
7,1.852600
8,1.777900
9,1.726900
10,1.746500


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

TrainOutput(global_step=20, training_loss=1.8304927110671998, metrics={'train_runtime': 81.2784, 'train_samples_per_second': 0.984, 'train_steps_per_second': 0.246, 'total_flos': 854323736641536.0, 'train_loss': 1.8304927110671998, 'epoch': 0.01})

In [ ]:
new_model = "my_mistral_7b"

trainer.model.save_pretrained(new_model)
model.config.use_cache = True

In [ ]:
trainer.model.push_to_hub(new_model, use_temp_dir=False)
model = model.merge_and_unload()

# model.push_to_hub(new_model, use_temp_dir=False)
# tokenizer.push_to_hub(new_model, use_temp_dir=False)

README.md:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
from peft import PeftModel
import torch
base_model_reload = AutoModelForCausalLM.from_pretrained(
        model_id,
        return_dict=True,
        quantization_config=bnb_config,
        low_cpu_mem_usage=True,
        device_map="auto",
        trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
model = PeftModel.from_pretrained(base_model_reload, new_model)

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyForCausalLM', 'MvpForCausalLM', 'OpenLlam

In [ ]:
prompt = "How become a DataCamp certified data professional"

sequences = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=100,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(sequences[0]['generated_text'])

<s>[INST] How become a DataCamp certified data professional [/INST] DataCamp is an interactive platform for learning data science. To become a DataCamp certified data professional, you can follow these steps:

1. Choose a track: DataCamp offers a variety of tracks, including Data Science, Data Engineering, Data Analytics, and more. Select a track that fits your career goals and skill level.

2. Complete the prerequisites: Each track has a list of prerequisites. Complete these before starting the track.

3


In [ ]:
model = model.merge_and_unload()

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/talktactapp/my_mistral_7b/commit/82ffebed02704784f7ec967d3941c84574ff2be0', commit_message='Upload tokenizer', commit_description='', oid='82ffebed02704784f7ec967d3941c84574ff2be0', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
!pip install accelerate -qqq


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
!pip install bitsandbytes -qqq


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 15.3 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline


pipe = pipeline(
    "text-generation",
    model = "talktactapp/my_mistral_7b",
    device_map="auto"
)

prompt = "How do I become a data engineer in 6 months?"

sequences = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=50,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,)

print(sequences[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] How do I become a data engineer in 6 months? [/INST] Becoming a data engineer in 6 months requires a focused and intentional approach. Here are some steps you can take to get started:

1. Assess your current skills: Before diving into data engineering, it's essential to


In [ ]:
prompt = "How can i find true love?"

sequences = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=200,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,)

print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] How can i find true love? [/INST] Finding true love is a personal journey that varies from person to person. Here are some general suggestions that might help you on your quest:

1. Self-love: Before you can truly love someone else, it's important to love yourself. This means taking care of your physical, emotional, and mental health. Practice self-care, engage in activities that bring you joy, and surround yourself with positive influences.
2. Set clear intentions: Be clear about what you're looking for in a partner and what your priorities are in a relationship. Reflect on past relationships and identify what worked and what didn't.
3. Expand your social circle: Put yourself out there and meet new people. Join clubs, attend social events, and engage in activities that align with your interests. The more people you meet, the greater your chances of finding someone who is compatible with you.
4. Be open-minded: Don't limit yourself to a certain type


In [ ]:
# User input and prompt
user_input = "can you tell me a sad story"
prompt = "Generate a response for the following prompt:"

# Combine user input and prompt
input_text = f"<s>[INST] {user_input} [INST]"

# Tokenize and encode the input
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generate response using the GPT-2 model with adjusted temperature and do_sample=True
output = model.generate(
    input_ids,
    max_length=100,
    temperature=0.8,
    top_k=50,
    top_p=0.95,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True  # Set do_sample to True
)

response = tokenizer.decode(output[0], skip_special_tokens=True)

print("User:", user_input)
print("Model Response:", response)

In [ ]:
# User input and prompt
user_input = "How can i find true love?"
prompt = "Generate a response for the following prompt:"

# Combine user input and prompt
input_text = f"{prompt} {user_input}"

# Tokenize and encode the input
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generate response using the GPT-2 model with adjusted temperature and do_sample=True
output = model.generate(
    input_ids,
    max_length=100,
    temperature=0.6,
    top_k=50,
    top_p=0.95,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True  # Set do_sample to True
)

response = tokenizer.decode(output[0], skip_special_tokens=True)

print("User:", user_input)
print("Model Response:", response)

In [ ]:
# User input and prompt
user_input = "How can i find true love?"
prompt = "Generate a response for the following prompt:"

# Combine user input and prompt
input_text = f"<s>[INST] {user_input} [INST]"

# Tokenize and encode the input
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generate response using the GPT-2 model with adjusted temperature and do_sample=True
output = model.generate(
    input_ids,
    max_length=100,
    temperature=0.6,
    top_k=50,
    top_p=0.95,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True  # Set do_sample to True
)

response = tokenizer.decode(output[0], skip_special_tokens=True)

print("User:", user_input)
print("Model Response:", response)

In [ ]:
# User input and prompt
user_input = "How can i find true love?"
prompt = "Generate a response for the following prompt:"

# Combine user input and prompt
input_text = f"<s>[INST] {user_input} [INST]"

# Tokenize and encode the input
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generate response using the GPT-2 model with adjusted temperature and do_sample=True
output = model.generate(
    input_ids,
    max_length=100,
    temperature=0.8,
    top_k=50,
    top_p=0.95,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True  # Set do_sample to True
)

response = tokenizer.decode(output[0], skip_special_tokens=True)

print("User:", user_input)
print("Model Response:", response)

In [ ]:
# Define the output directory for saving the model
output_dir = "/content/outputs"

# Save the model using the SFTTrainer's save_model method
trainer.save_model(output_dir)

# Compress the model directory into a zip file
import shutil
shutil.make_archive(output_dir, 'zip', output_dir)

In [ ]:
# User input and prompt
user_input = "How are you?"
prompt = "Generate a response for the following prompt:"

# Combine user input and prompt
input_text = f"{prompt} {user_input}"

# Tokenize and encode the input
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generate response using the GPT-2 model with adjusted temperature and do_sample=True
output = model.generate(
    input_ids,
    max_length=300,
    temperature=0.8,
    top_k=50,
    top_p=0.95,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True  # Set do_sample to True
)

response = tokenizer.decode(output[0], skip_special_tokens=True)

print("User:", user_input)
print("Model Response:", response)